In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, Model, regularizers
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
from tensorflow.keras.utils import plot_model
from sklearn.model_selection import GroupShuffleSplit
import pandas as pd




In [ ]:
!ls -lh /content


In [ ]:
!ls -lh /content/X.npy
!ls -lh /content/X_test.npy


In [ ]:
X = np.load("/content/X.npy")
print("Shape:", X.shape)
print("Dtype:", X.dtype)

x_test = np.load("/content/X_test.npy")
print("Shape:", x_test.shape)
print("Dtype:", x_test.dtype)


y = np.load("/content/y.npy")
print("Shape:", y.shape)
print("Dtype:", y.dtype)


y_test = np.load("/content/y_test.npy")
print("Shape:", y_test.shape)
print("Dtype:", y_test.dtype)

**Jagadish Scoring (48h)**



1.   +2 if max temp > 35°C
2.   +1 if max temp > 33°C
3.   +1 if rainfall < 2 mm
4.   +1 if ≥2 consecutive hot days (>33°C)
5.   +1 if mean VPD > 2.5 kPa

**Hybrid Label**

1. High (2): score ≥ 4 or deficit ≥ 40
2. Medium (1): score ≥ 2 or deficit ≥ 15
3. Low (0): otherwise

In [ ]:
site_ids = np.load("/content/site_ids.npy")
site_ids_test = np.load("/content/site_ids_test.npy")


In [ ]:
print("Data shapes:", site_ids.shape, site_ids_test.shape)

In [ ]:
n_features = X.shape[2]
n_classes = len(np.unique(y))

In [ ]:
n_total = len(X)

In [ ]:
gss = GroupShuffleSplit(test_size=0.2, random_state=42)
train_idx, val_idx = next(gss.split(X, y, groups=site_ids))

X_train = X[train_idx]
y_train = y[train_idx]
site_ids_train = site_ids[train_idx]

X_val   = X[val_idx]
y_val   = y[val_idx]
site_ids_val = site_ids[val_idx]


print("Train:", X_train.shape, y_train.shape , site_ids_train.shape)
print("Val:  ", X_val.shape, y_val.shape , site_ids_val.shape)
print("Test: ", x_test.shape, y_test.shape, site_ids_test.shape)

In [ ]:
print("Train:", X_train.shape, "Val:", X_val.shape, "Test:", x_test.shape)

In [ ]:
def normalize_per_location(X, site_ids):
    """
    Normalize each feature per location using mean and std computed over
    all samples and timesteps from that location.

    Parameters
    X : np.ndarray
        Shape (num_samples, timesteps, num_features)
    site_ids : np.ndarray
        Shape (num_samples,). Each entry is a string or int identifying the site.

    Returns
    X_norm : np.ndarray
        Normalized array of the same shape as X.
    stats : dict
        Dictionary of {location: (mean, std)} for reproducibility or test normalization.
    """
    X_norm = np.zeros_like(X)
    stats = {}

    for loc in np.unique(site_ids):
        idx = np.where(site_ids == loc)[0]
        loc_data = X[idx]

        mean = loc_data.mean(axis=(0, 1), keepdims=True)
        std = loc_data.std(axis=(0, 1), keepdims=True) + 1e-6

        X_norm[idx] = (loc_data - mean) / std
        stats[loc] = (mean, std)

    return X_norm, stats

In [ ]:
def test_normalize_per_location(X_test,site_ids_test):
    X_test_norm = np.zeros_like(X_test)
    for loc in np.unique(site_ids_test):
        idx = np.where(site_ids_test == loc)[0]
        loc_data = X_test[idx]

        if loc in stats:
            mean, std = stats[loc]
        else:
            mean = loc_data.mean(axis=(0, 1), keepdims=True)
            std  = loc_data.std(axis=(0, 1), keepdims=True) + 1e-6

        X_test_norm[idx] = (loc_data - mean) / std

    return X_test_norm


In [ ]:
X_train, stats = normalize_per_location(X_train, site_ids_train)
X_val, _ = normalize_per_location(X_val, site_ids_val)
x_test = test_normalize_per_location(x_test, site_ids_test)


**Baseline**

In [ ]:
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, classification_report
from sklearn.dummy import DummyClassifier
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Masking
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
import joblib


n_train = X_train.shape[0]
n_test  = x_test.shape[0]
Xtr_flat = X_train.reshape(n_train, -1)
Xte_flat = x_test.reshape(n_test, -1)

scaler = StandardScaler()
Xtr_flat_s = scaler.fit_transform(Xtr_flat)
Xte_flat_s = scaler.transform(Xte_flat)


print("\n=== Logistic Regression ===")
lr = LogisticRegression(max_iter=1000, multi_class="multinomial", solver="saga", C=1.0, n_jobs=-1)
lr.fit(Xtr_flat_s, y_train)
y_pred = lr.predict(Xte_flat_s)
print("LR - Acc:", accuracy_score(y_test, y_pred), "Macro-F1:", f1_score(y_test, y_pred, average="macro"))
print(classification_report(y_test, y_pred))
joblib.dump(lr, "baseline_lr.joblib")

print("\n=== Random Forest ===")
rf = RandomForestClassifier(n_estimators=300, max_depth=15, n_jobs=-1, random_state=42)
rf.fit(Xtr_flat, y_train)
y_pred = rf.predict(Xte_flat)
print("RF - Acc:", accuracy_score(y_test, y_pred), "Macro-F1:", f1_score(y_test, y_pred, average="macro"))
print(classification_report(y_test, y_pred))
joblib.dump(rf, "baseline_rf.joblib")



In [ ]:
print("\n=== 1-layer LSTM ===")
num_classes = len(np.unique(y))
ytr_cat = to_categorical(y_train, num_classes)
yte_cat = to_categorical(y_test, num_classes)

model1 = Sequential()
model1.add(Masking(mask_value=0., input_shape=(X_train.shape[1], X_train.shape[2])))
model1.add(LSTM(64, return_sequences=False))
model1.add(Dense(num_classes, activation="softmax"))
model1.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

es = EarlyStopping(monitor="val_loss", patience=6, restore_best_weights=True)
history = model1.fit(X_train, ytr_cat, validation_split=0.1, epochs=50, batch_size=64, callbacks=[es], verbose=2)
y_pred_proba = model1.predict(x_test)
y_pred_b = np.argmax(y_pred_proba, axis=1)
print("LSTM - Acc:", accuracy_score(y_test, y_pred_b), "Macro-F1:", f1_score(y_test, y_pred_b, average="macro"))
print(classification_report(y_test, y_pred_b))
model.save("baseline_lstm.h5")

joblib.dump(scaler, "scaler.joblib")

**Main Model**

In [ ]:
class SumPooling1D(layers.Layer):
    def call(self, inputs):
        return tf.reduce_sum(inputs, axis=1)

def attention_block(inputs):
    score = layers.Dense(1, activation="tanh", name="att_score")(inputs)
    weights = layers.Softmax(axis=1, name="att_weights")(score)  # ← NAME FIXED
    context = layers.Multiply(name="att_apply")([inputs, weights])
    context = SumPooling1D(name="att_context")(context)


    return context, weights



In [ ]:
def build_bilstm_attention_model(n_features, n_classes, timesteps=48):
    inputs = layers.Input(shape=(timesteps, n_features))
    x = layers.BatchNormalization()(inputs)

    x = layers.Bidirectional(
        layers.LSTM(64, return_sequences=True,
                    dropout=0.2, recurrent_dropout=0.2)
    )(x)

    x = layers.LSTM(32, return_sequences=True,
                    dropout=0.2, recurrent_dropout=0.2)(x)

    # attention
    context, attn_weights = attention_block(x)

    # classification head
    x = layers.Dense(32, activation="relu",
                     kernel_regularizer=regularizers.l2(1e-4))(context)
    x = layers.Dropout(0.3)(x)
    outputs = layers.Dense(n_classes, activation="softmax", name="pred")(x)

    model = Model(inputs, outputs=[outputs, attn_weights], name="BiLSTM_Attention")

    model.compile(
        optimizer=tf.keras.optimizers.Adam(1e-3),
        loss={"pred": "sparse_categorical_crossentropy", "att_weights": None},
        loss_weights={"pred": 1.0, "att_weights": 0.0},
        metrics={"pred": "accuracy"}
    )

    return model


In [ ]:
model = build_bilstm_attention_model(n_features, n_classes)

In [ ]:
model.output_names


In [ ]:
model.summary()

In [ ]:
cw = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
class_weights = {i: cw[i] for i in range(len(cw))}

In [ ]:

plot_model(
    model,
    to_file='bilstm_attention_model.png',
    show_shapes=True,
    show_layer_names=True,
    rankdir='TB',
    dpi=120
)

In [ ]:
callbacks = [
    keras.callbacks.EarlyStopping(
        monitor='val_loss',
        patience=6,
        restore_best_weights=True,
        verbose=1
    ),

    keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss',
        patience=3,
        factor=0.4,
        min_lr=1e-7,
        verbose=1
    )
]


history = model.fit(
    X_train,{"pred": y_train, "att_weights": y_train},
    validation_data=(X_val, {"pred": y_val, "att_weights": y_val}),
    epochs=40,
    batch_size=64,
    callbacks=callbacks

)


history_dict = history.history
train_acc = history_dict['pred_accuracy']
val_acc   = history_dict['val_pred_accuracy']
train_loss = history_dict['loss']
val_loss   = history_dict['val_loss']

epochs = range(1, len(train_acc) + 1)

plt.figure(figsize=(12,5))

plt.subplot(1, 2, 1)
plt.plot(epochs, train_acc, 'b-', label='Training Accuracy')
plt.plot(epochs, val_acc, 'r-', label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Training vs Validation Accuracy')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(epochs, train_loss, 'b-', label='Training Loss')
plt.plot(epochs, val_loss, 'r-', label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training vs Validation Loss')
plt.legend()

plt.tight_layout()
plt.show()


In [ ]:
history_dict = history.history
train_acc = history_dict['pred_accuracy']
val_acc   = history_dict['val_pred_accuracy']
train_loss = history_dict['loss']
val_loss   = history_dict['val_loss']

epochs = range(1, len(train_acc) + 1)

plt.figure(figsize=(12,5))

plt.subplot(1, 2, 1)
plt.plot(epochs, train_acc, 'b-', label='Training Accuracy')
plt.plot(epochs, val_acc, 'r-', label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Training vs Validation Accuracy')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(epochs, train_loss, 'b-', label='Training Loss')
plt.plot(epochs, val_loss, 'r-', label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training vs Validation Loss')
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
test_results = model.evaluate(
    x_test,
    {"pred": y_test, "att_weights": y_test},
    verbose=0
)

test_acc = test_results[-1]

print(f"\nTest Accuracy: {test_acc:.4f}")

model.save("bilstm_attention.keras")
model.save("bilstm_attention.h5")
model.save_weights("bilstm_attention.weights.h5")
print("Saved to content ")

from google.colab import files
files.download("bilstm_attention.h5")
files.download("bilstm_attention.keras")
files.download("bilstm_attention.weights.h5")
print("Downloaded to Local")

In [ ]:
y_pred, att_weights = model.predict(x_test[16:17])
att_weights = att_weights.squeeze()
plt.plot(att_weights)
plt.title("Attention over 48 hours")
plt.xlabel("Hour index (0–47)")
plt.ylabel("Importance")
plt.show()

In [ ]:
y_pred = model.predict(x_test)[0]
cm = confusion_matrix(y_test, np.argmax(y_pred, axis=1))
sns.heatmap(cm, annot=True, fmt='d')


In [ ]:
from sklearn.metrics import classification_report


y_pred_classes = np.argmax(y_pred, axis=1)

print(classification_report(y_test, y_pred_classes, digits=4))


In [ ]:
unique, counts = np.unique(y_train, return_counts=True)
pd.DataFrame({'Class': unique, 'Count': counts, 'Percent': counts / counts.sum() * 100})


Evaluation Metrics

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

from sklearn.metrics import (
    accuracy_score, f1_score, balanced_accuracy_score, matthews_corrcoef,
    log_loss, roc_auc_score, precision_recall_curve, auc,
    confusion_matrix
)

import matplotlib.pyplot as plt
import seaborn as sns


In [ ]:
y_proba = model.predict(x_test)
if isinstance(y_proba, (list, tuple)):
    y_proba = y_proba[0]
y_pred = np.argmax(y_proba, axis=1)
y_true = y_test

In [ ]:
acc = accuracy_score(y_true, y_pred)
macro_f1 = f1_score(y_true, y_pred, average='macro')
micro_f1 = f1_score(y_true, y_pred, average='micro')
balanced_acc = balanced_accuracy_score(y_true, y_pred)
mcc = matthews_corrcoef(y_true, y_pred)

print("Accuracy:", acc)
print("Macro-F1:", macro_f1)
print("Micro-F1:", micro_f1)
print("Balanced Accuracy:", balanced_acc)
print("MCC:", mcc)


In [ ]:
ll = log_loss(y_true, y_proba)
brier = np.mean(np.sum((y_proba - np.eye(3)[y_true])**2, axis=1))

print("Log Loss:", ll)
print("Brier Score:", brier)


In [ ]:
auc_macro = roc_auc_score(y_true, y_proba, multi_class='ovr', average='macro')
auc_micro = roc_auc_score(y_true, y_proba, multi_class='ovr', average='micro')

print("Macro ROC-AUC:", auc_macro)
print("Micro ROC-AUC:", auc_micro)


In [ ]:
pr_auc = {}
for c in range(3):
    y_bin = (y_true == c).astype(int)
    precision, recall, _ = precision_recall_curve(y_bin, y_proba[:, c])
    pr_auc[c] = auc(recall, precision)

print("PR-AUC:", pr_auc)


In [ ]:
cm = confusion_matrix(y_true, y_pred, normalize='true')

plt.figure(figsize=(6,5))
sns.heatmap(cm, annot=True, cmap='Blues', fmt='.2f')
plt.xlabel("Predicted")
plt.ylabel("True")
plt.title("Normalized Confusion Matrix")
plt.show()


In [ ]:
from sklearn.calibration import calibration_curve
def expected_calibration_error(y_true, y_proba, n_bins=15):
    probs = y_proba.max(axis=1)
    preds = np.argmax(y_proba, axis=1)
    bins = np.linspace(0.0, 1.0, n_bins + 1)
    ece = 0.0

    for i in range(n_bins):
        idx = (probs > bins[i]) & (probs <= bins[i+1])
        if np.any(idx):
            acc = np.mean(preds[idx] == y_true[idx])
            conf = np.mean(probs[idx])
            ece += (np.sum(idx) / len(y_true)) * np.abs(acc - conf)
    return ece

pred_conf = y_proba.max(axis=1)
pred_label = np.argmax(y_proba, axis=1)
correct = (pred_label == y_true).astype(int)

prob_true, prob_pred = calibration_curve(correct, pred_conf, n_bins=15)

plt.plot(prob_pred, prob_true, marker='o')
plt.plot([0,1],[0,1],'k--')
plt.xlabel("Predicted Probability")
plt.ylabel("Observed Frequency")
plt.title("Calibration Curve")
plt.show()

print("ECE:", expected_calibration_error(y_true, y_proba))


In [ ]:
def bootstrap_ci(y_true, y_pred, metric, n=2000):
    stats = []
    N = len(y_true)
    for _ in range(n):
        idx = np.random.randint(0, N, N)
        stats.append(metric(y_true[idx], y_pred[idx]))
    return np.percentile(stats, [2.5, 97.5])

acc_ci = bootstrap_ci(np.array(y_true), np.array(y_pred), accuracy_score)
f1_ci = bootstrap_ci(np.array(y_true), np.array(y_pred),
                     lambda a, b: f1_score(a, b, average='macro'))

print("Accuracy 95% CI:", acc_ci)
print("Macro-F1 95% CI:", f1_ci)


In [ ]:
from statsmodels.stats.contingency_tables import mcnemar

tb = np.zeros((2,2))

for yt, yp_b, yp_o in zip(y_true, y_pred_b, y_pred):
    tb[int(yp_b == yt)][int(yp_o == yt)] += 1

result = mcnemar(tb, exact=False, correction=True)
print("McNemar p-value:", result.pvalue)
print("Contingency table:\n", tb)
